In [1]:
import json
import numpy as np
from random import choice
from klm.query import LM, get_unigram_probs
from code.all import get_stops
from klm.query import slor

stop_words = get_stops()

comments = []
with open("corpora/data_ccrit.spacy.jsonl", "r") as inf:
    for i in inf:
        i = json.loads(i)
        comments.append(i)

DEV = {
        'klm_model': "/Users/ahandler/research/snippetbox/klm/all_gw.binary",
        'ug_model': "/Users/ahandler/research/snippetbox/klm/all_gw.unigrams.json"
}

env = DEV
LOC = env['klm_model']
UG_MODEL = env["ug_model"]

lm = LM(loc=LOC)
up = get_unigram_probs(UG_MODEL)

In [11]:
ix = 13

import re

def print_me(ix, o):
    return o if ix == 1 else "..."

def clean_dots(str_):
    a = "[... ]{2,8}"
    b = "\.{6,}"
    fix_multi_dots = re.sub(a, "...", str_)
    remove_long_dots = re.sub(b, "...", fix_multi_dots)
    return remove_long_dots


ops = []

for oo in range(1000):

    included = np.ones(len(comments[ix]["tokens"]))

    for o in range(choice([1,2])):

        d1 = choice(range(len(comments[ix]["tokens"])))
        included[d1] = 0
        deleted = choice(range(10))

        for i in range(deleted):
            if i + d1 < len(included):
                included[i + d1] = 0
    
    
    comment = [print_me(ix, o) for ix, o in zip(included, comments[ix]["tokens"])]
    
    sequence = " ".join([o for o in comment if o != "..."])
    

    remove_long_dots = clean_dots(" ".join(comment)
    
    worst = 1000
    for seq in remove_long_dots.split("..."):
        if slor(seq, lm, up) < worst:
            worst = slor(seq, lm, up)
    
    ops.append((remove_long_dots, worst))

ops.sort(key=lambda x:x[1], reverse=True)
print("***")
print(ops[0:4])
print(" ".join(comments[ix]["tokens"]), slor(" ".join(comments[ix]["tokens"]), lm, up))

***
[('I think it ... ... ... ... ... ... it would be a landmark ... ... ... ... ... ... ...', 2.9113133452499387), ('I think it would ... ... ... ... as it would be ... ... ... ... ... ... ... ... ...', 2.774708739130859), ('I think it would bring ... ... ... ... ... ... ... ... ... that represents a border ... ... ...', 2.769765535656738), ('I think ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... two cities', 2.7176468549092614)]
I think it would bring the community together as it would be a landmark that represents a border between two cities 1.5216273063848589


In [8]:
lm.score("bring the community", verbose=True)
slor("it would be a", lm, up)

-4.038015842437744 2: <s> bring
-1.0804628133773804 3: <s> bring the
-2.8131766319274902 3: bring the community
-0.8136436343193054 3: the community </s>


0.9955547498046875